In [34]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
from bs4 import BeautifulSoup

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.getenv("API_KEY"))

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
import re
# ヤフーニュースのURL
URL = "https://news.yahoo.co.jp/"

# ページのHTMLを取得
response_top = requests.get(URL)

# BeautifulSoupを使ってHTMLを解析
soup_top = BeautifulSoup(response_top.text, "html.parser")

# 最初のトピックスを取得
first_news_topic_url = soup_top.find("section", id="uamods-topics").find("li").find("a").get("href")

# 最初のトピックスの記事に移動
response_topic = requests.get(first_news_topic_url)
soup_topics = BeautifulSoup(response_topic.text, "html.parser")
article_url = soup_topics.find("a", attrs={"data-cl-params": "_cl_vmodule:tpc_main;_cl_link:headline;_cl_position:2;"}).get("href")

# 最初のトピックスの記事に移動
response_article = requests.get(article_url)
soup_article = BeautifulSoup(response_article.text, "html.parser")

# 記事のタイトルを取得
article_title = soup_article.find(id="uamods").find("h1").text

# 記事の本文を取得
article_body = soup_article.find("div", class_="article_body")
article_text = "\n".join([c.get_text() for c in article_body.find_all('p')])
print(article_text)

https://news.yahoo.co.jp/pickup/6538756
https://news.yahoo.co.jp/articles/60ae71cddafc8773e9020b1e2ff94cbb35c221f4
ジュネーブで記者会見するベッセント米財務長官（右）と米通商代表部（ＵＳＴＲ）のグリア代表（１２日）＝ロイター
　【ワシントン＝田中宏幸、北京＝照沼亮介】米中両国は日本時間の１４日午後１時１分（米国時間午前０時１分、中国時間午後０時１分）、相互に課している追加関税をそれぞれ１１５％引き下げた。１０～１１日にスイスで行われた米中閣僚級協議での合意に基づく措置で、今後９０日間、追加関税率は米国が３０％、中国が１０％となる。

【グラフ】ダウ平均（３０種）と日経平均株価（２２５種）の推移
　４月以降に米中双方が課した追加関税３４％のうち２４％分を９０日間停止し、上乗せした報復関税９１％分は互いに撤廃する。一方、米中双方が税率１０％を維持するほか、米国は合成麻薬フェンタニルの流入対策を名目に発動している２０％は引き続き課し、対中追加関税は合わせて３０％となる。これとは別に米国による中国製の自動車や鉄鋼・アルミニウムなどに対する品目別の追加関税は、引き下げの対象外となる。
　中国も、米国からの液化天然ガス（ＬＮＧ）や大豆など一部の輸入品に対する最大１５％の報復関税は合意の対象外として維持する。
　米ブルームバーグ通信によると、中国政府は１３日までに国内の航空会社に要請していた米ボーイング社製航空機の納入停止を解除した。スイスでの米中合意には、中国が関税以外の報復措置の一時停止か廃止をすることが盛り込まれ、中国側が対応した格好だ。
　トランプ米大統領は１２日、「習近平（シージンピン）国家主席と話すつもりだ。恐らく週末になる」と述べており、米中首脳間の電話などでの協議がいつ実現するかが次の焦点となる。


In [61]:
# 記事の要約を行うプロンプト作成
prompt = f"""
以下の文章を要約してください

# 条件
- 100文字以内

# 文章
{article_text[:2000]}
"""

# APIを使って要約を生成
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=300,
    temperature=0.3
)

In [63]:
# LLMからの回答を表示
print(f"ヤフーニュースの一番上のトピックは「{article_title}」です。")
print(response.choices[0].message.content)

ヤフーニュースの一番上のトピックは「米中双方が関税１１５％引き下げ…追加関税は米国３０％・中国１０％に、スイス合意に基づき」です。
米中は相互の追加関税を115％引き下げ、今後90日間は米国30％、中国10％に。合意には報復措置の一時停止も含まれ、米中首脳の協議が注目されている。
